# More in Depth Example on Functional API

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, regularizers
import pandas as pd
import os

### Hyperparameters

In [2]:
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001

### Dataset
【🔔】[**Download**](https://www.kaggle.com/datasets/eb9594e5b728b2eb74ff8d5e57a9b74634330bfa79d9195d6ebdc7745b9802c3)

In [3]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_images = os.getcwd() + "/train_images/" + train_df.iloc[:,0].values
test_images = os.getcwd() + "/test_images/" + test_df.iloc[:,0].values

train_labels = train_df.iloc[:,1:].values
test_labels = test_df.iloc[:,1:].values

In [4]:
def read_images(images_path, label):
    image = tf.io.read_file(images_path)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)
    
    image.set_shape((64, 64, 1))
    label[0].set_shape([])
    label[1].set_shape([])
    
    labels = {"first_num":label[0], "second_num":label[1]}
    return image, labels

In [5]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = tf.data.Dataset.from_tensor_slices(
    (train_images, train_labels)
)
train_dataset = (
    train_dataset.shuffle(buffer_size=len(train_labels))
    .map(read_images)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)
test_dataset = tf.data.Dataset.from_tensor_slices(
    (test_images, test_labels)
)
test_dataset = (
    test_dataset
    .map(read_images)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

### Model

In [6]:
inputs = keras.Input(shape=(64, 64, 1))
x = layers.Conv2D(
    filters = 32,
    kernel_size = 3,
    padding = 'same',
    kernel_regularizer = regularizers.l2(WEIGHT_DECAY)
)(inputs)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.Conv2D(64, 3, kernel_regularizer = regularizers.l2(WEIGHT_DECAY))(x)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3, activation='relu',
                  kernel_regularizer = regularizers.l2(WEIGHT_DECAY))(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation='relu')(x)
output1 = layers.Dense(10, activation='softmax', name='first_num')(x)
output2 = layers.Dense(10, activation='softmax', name='second_num')(x)
model = keras.Model(inputs = inputs, outputs = [output1, output2])
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 64, 64, 32)   320         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 64, 64, 32)  128         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 tf.nn.relu (TFOpLambda)        (None, 64, 64, 32)   0           ['batch_normalization[0][0]']

In [7]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=[
            keras.losses.SparseCategoricalCrossentropy(),
            keras.losses.SparseCategoricalCrossentropy(),
        ],
    metrics=["accuracy"]
)

model.fit(train_dataset, epochs=5, verbose=2)
model.evaluate(test_dataset, verbose=2)

Epoch 1/5
1000/1000 - 1300s - loss: 1.8437 - first_num_loss: 0.8805 - second_num_loss: 0.8798 - first_num_accuracy: 0.6845 - second_num_accuracy: 0.6862 - 1300s/epoch - 1s/step
Epoch 2/5
1000/1000 - 1373s - loss: 0.4805 - first_num_loss: 0.2075 - second_num_loss: 0.2074 - first_num_accuracy: 0.9342 - second_num_accuracy: 0.9330 - 1373s/epoch - 1s/step
Epoch 3/5
1000/1000 - 1332s - loss: 0.3135 - first_num_loss: 0.1260 - second_num_loss: 0.1310 - first_num_accuracy: 0.9612 - second_num_accuracy: 0.9587 - 1332s/epoch - 1s/step
Epoch 4/5
1000/1000 - 1258s - loss: 0.2460 - first_num_loss: 0.0983 - second_num_loss: 0.0967 - first_num_accuracy: 0.9692 - second_num_accuracy: 0.9687 - 1258s/epoch - 1s/step
Epoch 5/5
1000/1000 - 1540s - loss: 0.2063 - first_num_loss: 0.0816 - second_num_loss: 0.0768 - first_num_accuracy: 0.9737 - second_num_accuracy: 0.9757 - 1540s/epoch - 2s/step
313/313 - 119s - loss: 0.9784 - first_num_loss: 0.3663 - second_num_loss: 0.5660 - first_num_accuracy: 0.9027 - sec

[0.978449821472168,
 0.3662666380405426,
 0.5659745335578918,
 0.9027000069618225,
 0.8529499769210815]